In [30]:
from sklearn.cluster import KMeans


In [450]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.cluster import KMeans

# import data
drivers = pd.read_csv("driver_ids.csv",parse_dates = ["driver_onboard_date"])
ride = pd.read_csv("ride_ids.csv")
time = pd.read_csv("ride_timestamps.csv",parse_dates = ["timestamp"])
# convert timestamp table to pivot table to aid with calculating difference later
time_flat = time.pivot(index = "ride_id",columns = "event", values = "timestamp")
# merge above time flat table with ride table
time_ride = time_flat.merge(ride,how = "inner",on = "ride_id")
time_ride = time_ride.sort_values(["driver_id","accepted_at"])
# merge with drivers table
out = time_ride.merge(drivers,how = "inner",on = "driver_id")
# out is final table, contains drivers information, ride information and timestamps information
# calculate every ride value
out['value'] =(1.75 + 2 + out['ride_distance']*0.000621371*1.15 + out['ride_duration']/60*0.22)*(1+out['ride_prime_time']/100)

In [337]:
out.loc[out['value']<=5,'value'] = 5
out.loc[out['value']>=400,'value'] = 400
out['value'].min()

5.0

In [451]:
# calculate the week that each ride happens for each driver
out['week'] = (out['picked_up_at'] - out['driver_onboard_date'].min()).dt.days//7
# extract onboarding month information
out['onboard_m'] = pd.DatetimeIndex(out['driver_onboard_date']).month

**calculate recency for each driver**


In [452]:
# calculate the recency
# below calculate the recency by days
recency = out.groupby('driver_id')['arrived_at'].max().reset_index()
recency.columns = ['driver_id','max_ride_time']
recency['max_ride_time'].max()
recency['recency'] = (recency['max_ride_time'].max()-recency['max_ride_time']).dt.days
recency = recency[['driver_id','recency']]

**retrieve march, apr, may cohorts**

In [454]:
cohortMar = out[out['onboard_m'] == 3]
cohortMar['week'] = (cohortMar['picked_up_at'] - cohortMar['driver_onboard_date'].min()).dt.days//7
cohortApr = out[out['onboard_m'] == 4]
cohortApr['week'] = (cohortApr['picked_up_at'] - cohortApr['driver_onboard_date'].min()).dt.days//7
cohortMay = out[out['onboard_m'] == 5]
cohortMay['week'] = (cohortMay['picked_up_at'] - cohortMay['driver_onboard_date'].min()).dt.days//7

C:\Users\tyjse\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\tyjse\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\tyjse\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [55]:
# from tslearn.clustering import TimeSeriesKMeans
# weekly_count = cohortMar.groupby(['driver_id','week'])
# weekly_count = weekly_count['ride_id'].count().reset_index()
# weekly_count = weekly_count.pivot(index = "driver_id",columns = "week", values = "ride_id")
# weekly_count.fillna(0, inplace = True)
# model = TimeSeriesKMeans(n_clusters=3, metric="dtw")
# model.fit(weekly_count)
# pred = model.predict(weekly_count)
# Count = pd.DataFrame(weekly_count)
# Count['count_cluster'] = pred
# Count = Count.reset_index()
# Count.head()

**1. Calculate weekly driving count**

In [455]:
weekly_count_mar = cohortMar.groupby(['driver_id','week'])
weekly_count_mar = weekly_count_mar['ride_id'].count().reset_index()
weekly_count_mar = weekly_count_mar.pivot(index = "driver_id",columns = "week", values = "ride_id")
weekly_count_mar.fillna(0, inplace = True)
# kmeans = KMeans(n_clusters=3, init='k-means++')
# kmeans.fit(weekly_count)
# pred = kmeans.predict(weekly_count)
# Count = pd.DataFrame(weekly_count)
# Count['count_cluster'] = pred
# Count = Count.reset_index()
# Count['count_mean'] = Count.iloc[:,2:14].mean(axis=1)
# Count

**2. Calculate weekly driving value generated**

In [456]:
weekly_value_mar = cohortMar.groupby(['driver_id','week'])
weekly_value_mar = weekly_value_mar['value'].sum().reset_index()
weekly_value_mar = weekly_value_mar.pivot(index = "driver_id",columns = "week", values = "value")
weekly_value_mar.fillna(0, inplace = True)
# kmeans = KMeans(n_clusters=3, init='k-means++')
# kmeans.fit(weekly_value)
# pred = kmeans.predict(weekly_value)
# Value = pd.DataFrame(weekly_value)
# Value['value_cluster'] = pred
# Value = Value.reset_index()
# Value['value_mean'] = Value.iloc[:,2:14].mean(axis = 1)
# Value

**3. merge value and count table, recency table**

In [457]:
c_v_mar = recency.merge(weekly_count_mar, how = "inner", on = 'driver_id')
c_v_mar = c_v_mar.merge(weekly_value_mar, how = 'inner', on = 'driver_id')
c_v_mar.set_index('driver_id',inplace = True)

**4. run kmeans on the c_v dataset**

In [458]:
kmeans = KMeans(n_clusters=3, init='k-means++')
kmeans.fit(c_v)
pred = kmeans.predict(c_v)
Cluster = pd.DataFrame(c_v)
Cluster['cluster'] = pred
Cluster = Cluster.reset_index()

**5. run kmeans on centalized c_v data**

In [459]:
c_v_mar = recency.merge(weekly_count_mar, how = "inner", on = 'driver_id')
c_v_mar = c_v_mar.merge(weekly_value_mar, how = 'inner', on = 'driver_id')
c_v_mar.set_index('driver_id',inplace = True)
origin_mar = recency.merge(weekly_count_mar, how = "inner", on = 'driver_id')
origin_mar = origin_mar. merge(weekly_value_mar, how = 'inner', on = 'driver_id')
origin_mar.set_index('driver_id',inplace = True)
for i in c_v_mar.columns:
    c_v_mar[i] = (c_v_mar[i] - c_v_mar[i].mean())/c_v_mar[i].std()
import random
random.seed(2020)
kmeans = KMeans(n_clusters=3, init='k-means++')
kmeans.fit(c_v_mar)
pred = kmeans.predict(c_v_mar)
Cluster_mar = pd.DataFrame(c_v_mar)
Cluster_mar['cluster'] = pred
Cluster_mar = Cluster_mar.reset_index()
Cluster_mar = Cluster_mar[['driver_id','cluster']]
Cluster_mar = Cluster_mar.merge(origin_mar, how = "inner", on = 'driver_id')

**6. calculate some matrics**

In [460]:
Cluster_mar['valueq10'] = Cluster_mar.iloc[:,16:29].quantile(q=0.1,axis = 1)
Cluster_mar['valueq90'] = Cluster_mar.iloc[:,16:29].quantile(q=0.9,axis = 1)
Cluster_mar['valueq50'] = Cluster_mar.iloc[:,16:29].quantile(q=0.5,axis = 1)
Cluster_mar['value_mean'] = Cluster_mar.iloc[:,16:29].sum(axis = 1)/13
sns.scatterplot(data = Cluster_mar, x ='valueq10', y = 'value_mean', hue = 'cluster')

In [461]:
sns.scatterplot(data = Cluster_mar, x ='valueq10', y = 'valueq90', hue = 'cluster')

In [462]:
#Cluster_mar[Cluster_mar['cluster'] ==2].iloc[:,0:16]

Based on the normalized results, the group classified as 0, is the possible churn group;
the group classified as 1, there is potential to churn, and weekly drive count is not consistent
the group classified as 2, there is small possibility to churn

**7. Running through cohort Apr and May**

a. Apr

In [463]:
weekly_count_apr = cohortApr.groupby(['driver_id','week'])
weekly_count_apr = weekly_count_apr['ride_id'].count().reset_index()
weekly_count_apr = weekly_count_apr.pivot(index = "driver_id",columns = "week", values = "ride_id")
weekly_count_apr.fillna(0, inplace = True)
weekly_value_apr = cohortApr.groupby(['driver_id','week'])
weekly_value_apr = weekly_value_apr['value'].sum().reset_index()
weekly_value_apr = weekly_value_apr.pivot(index = "driver_id",columns = "week", values = "value")
weekly_value_apr.fillna(0, inplace = True)
c_v_apr = recency.merge(weekly_count_apr, how = "inner", on = 'driver_id')
c_v_apr = c_v_apr.merge(weekly_value_apr, how = 'inner', on = 'driver_id')
c_v_apr.set_index('driver_id',inplace = True)
origin_apr = recency.merge(weekly_count_apr, how = "inner", on = 'driver_id')
origin_apr = origin_apr. merge(weekly_value_apr, how = 'inner', on = 'driver_id')
origin_apr.set_index('driver_id',inplace = True)
for i in c_v_apr.columns:
    c_v_apr[i] = (c_v_apr[i] - c_v_apr[i].mean())/c_v_apr[i].std()
random.seed(2021)
kmeans = KMeans(n_clusters=3, init='k-means++')
kmeans.fit(c_v_apr)
pred = kmeans.predict(c_v_apr)
Cluster_apr = pd.DataFrame(c_v_apr)
Cluster_apr['cluster'] = pred
Cluster_apr = Cluster_apr.reset_index()
Cluster_apr = Cluster_apr[['driver_id','cluster']]
Cluster_apr = Cluster_apr.merge(origin_apr, how = "inner", on = 'driver_id')
Cluster_apr['valueq10'] = Cluster_apr.iloc[:,16:29].quantile(q=0.1,axis = 1)
Cluster_apr['valueq90'] = Cluster_apr.iloc[:,16:29].quantile(q=0.9,axis = 1)
Cluster_apr['valueq50'] = Cluster_apr.iloc[:,16:29].quantile(q=0.5,axis = 1)
Cluster_apr['value_mean'] = Cluster_apr.iloc[:,16:29].sum(axis = 1)/12
#sns.scatterplot(data = Cluster_apr, x ='valueq10', y = 'valueq90', hue = 'cluster')

b. May cohort

In [464]:
weekly_count_may = cohortApr.groupby(['driver_id','week'])
weekly_count_may = weekly_count_may['ride_id'].count().reset_index()
weekly_count_may = weekly_count_may.pivot(index = "driver_id",columns = "week", values = "ride_id")
weekly_count_may.fillna(0, inplace = True)
weekly_value_may = cohortApr.groupby(['driver_id','week'])
weekly_value_may = weekly_value_may['value'].sum().reset_index()
weekly_value_may = weekly_value_may.pivot(index = "driver_id",columns = "week", values = "value")
weekly_value_may.fillna(0, inplace = True)
c_v_may = recency.merge(weekly_count_may, how = "inner", on = 'driver_id')
c_v_may = c_v_may.merge(weekly_value_may, how = 'inner', on = 'driver_id')
c_v_may.set_index('driver_id',inplace = True)
origin_may = recency.merge(weekly_count_may, how = "inner", on = 'driver_id')
origin_may = origin_may. merge(weekly_value_may, how = 'inner', on = 'driver_id')
origin_may.set_index('driver_id',inplace = True)
for i in c_v_may.columns:
    c_v_may[i] = (c_v_may[i] - c_v_may[i].mean())/c_v_may[i].std()
random.seed(2020)
kmeans = KMeans(n_clusters=3, init='k-means++')
kmeans.fit(c_v_may)
pred = kmeans.predict(c_v_may)
Cluster_may = pd.DataFrame(c_v_may)
Cluster_may['cluster'] = pred
Cluster_may = Cluster_may.reset_index()
Cluster_may = Cluster_may[['driver_id','cluster']]
Cluster_may = Cluster_may.merge(origin_may, how = "inner", on = 'driver_id')
Cluster_may['valueq10'] = Cluster_may.iloc[:,16:29].quantile(q=0.1,axis = 1)
Cluster_may['valueq90'] = Cluster_may.iloc[:,16:29].quantile(q=0.9,axis = 1)
Cluster_may['valueq50'] = Cluster_may.iloc[:,16:29].quantile(q=0.5,axis = 1)
Cluster_may['value_mean'] = Cluster_may.iloc[:,16:29].sum(axis = 1)/12
#sns.scatterplot(data = Cluster_may, x ='valueq10', y = 'valueq90', hue = 'cluster')

In [465]:
#Cluster_apr[Cluster_apr['cluster'] == 1].iloc[:,0:18]

**8. put three cohorts together**

In [466]:
a = Cluster_mar[['driver_id','cluster','valueq10','valueq90','value_mean','recency']]
b = Cluster_apr[['driver_id','cluster','valueq10','valueq90','value_mean','recency']]
c = Cluster_may[['driver_id','cluster','valueq10','valueq90','value_mean','recency']]
a= a.append(b)
churn = a.append(c)
#sns.scatterplot(data = churn, x ='valueq10', y = 'valueq90', hue = 'cluster')

In [467]:
#sns.scatterplot(data = churn, x ='recency', y = 'valueq90', hue = 'cluster')

**9. Compute some other factors relevant to churn behavior**

a. First week rides and value

In [468]:
out['first_week'] = out['driver_onboard_date'] + timedelta(days=7)
out_first_week = out[out['accepted_at']<= out['first_week']]
out_first_week_count = out_first_week.groupby('driver_id').ride_id.count().reset_index()
out_first_week_count.rename(columns = {'ride_id':'first_week_frequency'}, inplace = True)
first_week_value = out_first_week.groupby('driver_id').value.sum().reset_index()
first_week_value.rename(columns = {'value':'first_week_value'}, inplace = True)
#merge with churn data set
churn = churn.merge(out_first_week_count, how = "inner", on = "driver_id")
churn = churn.merge(first_week_value, how = "inner", on = "driver_id")


In [469]:
#sns.scatterplot(data = churn, x ='first_week_frequency', y = 'first_week_value', hue = 'cluster')

check the boxplot

In [470]:
#sns.boxplot(y='first_week_frequency', x='cluster', data=churn)

In [471]:
#sns.boxplot(y='first_week_value', x='cluster', data=churn)

c. calculate driver response time

In [211]:
cohortMar['response'] = (cohortMar['accepted_at'] - cohortMar['requested_at']).dt.seconds
# calculate means response for each driver
response_time = cohortMar.groupby('driver_id').response.mean().reset_index()
Cluster2 = Cluster2.merge(response_time, how = 'left', on = 'driver_id')

C:\Users\tyjse\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
